# 実習の説明


## 何をするか？

非小細胞肺癌 (non-small-cell lung cancer; NSCLC) において、腫瘍組織内に存在する間質細胞から腫瘍細胞にシグナルのクロストークが送って腫瘍が活性化される可能性を探ります。データは、NSCLCモデルマウスと野生型のマウスの肺から、セルソーターで分けたマクロファージ、単球細胞、好中球、上皮細胞のRNA-seqデータです。

> このようなRNA-seqデータを、「変数が全遺伝子、条件が細胞型xマウス（野生型 or 腫瘍）、値が発現量である表」として想像できると、あとの解析がスムーズです

元ネタはこちら: 

- Toi _et al_., Transcriptome Analysis of Individual Stromal Cell Populations Identifies Stroma-Tumor Crosstalk in Mouse Lung Cancer Model, Cell Reports (2015) http://dx.doi.org/10.1016/j.celrep.2015.01.040 

ソフトウェアやアノテーションデータが著者らのウェブページで公開されています。また、NGSデータの生データや処理済みデータはGene Expression Omnibus (GEO)より公開されています。

- ソフトウェアやアノテーションデータ http://209.160.41.231/u54/CCCExplorer/
- Gene Expression Omnibus (GEO)のNGSデータのページ https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE59831

## どうやるか？

腫瘍組織内間質細胞のリガンドから腫瘍細胞の受容体へのシグナルのクロストークがあるとしたら、

- 腫瘍モデルマウスでは野生型に比べ、リガンドが発現上昇しているだろう
- 腫瘍モデルマウスの上皮組織では、そのリガンドの受容体が発現しているだろう

そのようなリガンドと受容体のペアを見つけるのに必要なデータは？

- 既知のリガンド-受容体ペアのデータベース（ヒト）
- ヒト-マウスオーソログの対応表
- 腫瘍組織内間質細胞で発現上昇がみられた遺伝子のリスト
- 腫瘍細胞で発現上昇

それらのデータを組み合わせればよさそう

![image.png](https://ars.els-cdn.com/content/image/1-s2.0-S2211124715000650-fx1.jpg)
(Image from http://dx.doi.org/10.1016/j.celrep.2015.01.040)

# **データの読み込みと整形**

４種類のテーブルデータを読み込みます。

## ライブラリのロード
ライブラリを使えるようにするためにロードする

In [1]:
import pandas as pd
import numpy as np
import math

## テーブルデータ１：遺伝子発現変動解析の結果のデータを眺めてみる

遺伝子はたくさんありますが、その中でも、「発現量がサンプル間で変動している遺伝子」に注意が向きます。なぜなら、そのような遺伝子は、サンプル間の表現型の違いを説明する可能性があるからです。

そのような遺伝子は一般に発現変動遺伝子（Differentially expressed genes; DEG） と呼ばれます。

ここでは正常組織と腫瘍組織のマクロファージの発現変動遺伝子解析の結果のデータを読み込んで、値の分布などを調べます。

- 詳しく言うと、正常組織と腫瘍組織のマクロファージから取得した RNA-seq データから、正常組織と腫瘍組織の間で発現が変動しているかを全ての遺伝子について網羅的に仮説検定した結果のデータ (`data/CCCExplorer/CD11CB_output/gene_exp.diff`) です。
 
これを例に、データの眺め方をみていきます。

In [2]:
# データの読み込み
url_deg_macrophage ="https://raw.githubusercontent.com/bioinfo-tsukuba/AdvancedCourse2021/main/3/data/CCCExplorer/CD11CB_output/gene_exp.diff"
deg_macrophage = pd.read_csv(url_deg_macrophage, sep="\t")

In [3]:
# 行数、何数を調べる
deg_macrophage.shape

(30743, 14)

In [4]:
# データの頭の部分だけ確認
deg_macrophage.head()

,test_id,gene_id,gene,locus,sample_1,sample_2,status,value_1,value_2,log2(fold_change),test_stat,p_value,q_value,significant
0,0610005C13Rik,0610005C13Rik,-,chr7:45567794-45589710,q1,q2,NOTEST,0.072521,0.044282,-0.711684,0.000000,1.00000,1.000000,no
1,0610007C21Rik,0610007C21Rik,-,chr5:31036035-31054623,q1,q2,OK,50.268700,47.849600,-0.071153,-0.229939,0.71855,0.839597,no
2,0610007N19Rik,0610007N19Rik,-,chr15:32240567-32244662,q1,q2,NOTEST,0.238515,0.062952,-1.921740,0.000000,1.00000,1.000000,no
3,0610007P08Rik,0610007P08Rik,-,chr13:63815319-63900301,q1,q2,OK,3.574360,3.112430,-0.199643,-0.579890,0.36160,0.542589,no
4,0610007P14Rik,0610007P14Rik,-,chr12:85815454-85824545,q1,q2,OK,11.635600,10.572300,-0.138247,-0.474073,0.54570,0.711416,no


実はこのようなデータでした

- 縦に遺伝子名のようなものが並んでいる。各行が遺伝子、各列が変数になっている
- `p_value`や`q_value`は発現変動を判定する統計検定の結果だろう
- `log2(fold_change)` は `value_2`/`value_1`をlog2変換したものだろう
    - ちなみに、`value_1` は正常組織でのマクロファージ、 `value_2` は腫瘍組織内マクロファージ (intratumoral macropahges)でのFPKMの平均値
      - これは __メタデータ__ をみないとわからない

In [5]:
# 使いやすいように、使いたい列だけにする
deg_macrophage = deg_macrophage[['gene_id', 'value_1', 'value_2', 'log2(fold_change)', 'p_value', 'q_value']]

In [6]:
# データの形が変わったことを確認
deg_macrophage.head()

,gene_id,value_1,value_2,log2(fold_change),p_value,q_value
0,0610005C13Rik,0.072521,0.044282,-0.711684,1.00000,1.000000
1,0610007C21Rik,50.268700,47.849600,-0.071153,0.71855,0.839597
2,0610007N19Rik,0.238515,0.062952,-1.921740,1.00000,1.000000
3,0610007P08Rik,3.574360,3.112430,-0.199643,0.36160,0.542589
4,0610007P14Rik,11.635600,10.572300,-0.138247,0.54570,0.711416


#### 発現変動遺伝子の数を数える

遺伝子はたくさんありますが、その中でも、「発現量がサンプル間で変動している遺伝子」に注意が向きます。なぜなら、そのような遺伝子は、サンプル間の表現型の違いを説明する可能性があるからです。

そのような遺伝子は一般に発現変動遺伝子（Differentially expressed genes; DEG） と呼ばれます。ここでは、発現変動遺伝子を抽出することを試みます。



In [7]:
# Fold change (fc) と　q-value (qv) の閾値を設定する

threshold_fc = math.log2(1.5)
threshold_qv = 0.1 

In [8]:
# 腫瘍組織内マクロファージで多い (FC > 1.5, q_value < 0.1) 遺伝子の数は？
deg_macrophage.query('`log2(fold_change)` > @threshold_fc & q_value < @threshold_qv').shape

(2324, 6)

In [9]:
# 正常組織内マクロファージで多い (FC > 1/1.5, q_value < 0.1) の遺伝子の数は？
deg_macrophage.query('`log2(fold_change)` > -1 * @threshold_fc & q_value < @threshold_qv').shape

(3334, 6)

#### ポジコンが取れているかを確認する
一般に実験がうまくいったかを確認するためには、「その実験がうまくいっているとすると絶対に観察される結果」を事前に設定します。これをポジコン（Positive control）と呼びます。

RNA-seq解析では、「このサンプルだとこの遺伝子は絶対に出ているはず」「この２サンプル群間でこの遺伝子は絶対に発現量に差が出ているはず」という遺伝子がポジコンに当たります。

今回は、「腫瘍内マクロファージでは、正常組織でのマクロファージに比べ、MMP-9やVEGFA遺伝子の発現量が高くなっている」という先行研究を参照し（下記）、これらの遺伝子が「腫瘍内マクロファージで高いと判定される発現変動遺伝子」に含まれるかを確認します。これにより、このRNA-seqデータが、生物学的なシグナルを反映してるかを判断します。


> "intratumoral macrophages express increased cathepsin K, COX-2, MMP-9, PDGF-B, uPA, VEGFA, and HGF (Wang et al., 2011)"
    - Wang et al., 2011 http://www.sciencedirect.com/science/article/pii/S0169500211002546

In [10]:
# ポジコンである Vegfa がとれるかの確認 (生物学的なQC)
deg_macrophage.query('`log2(fold_change)` > @threshold_fc & q_value < @threshold_qv').query('gene_id=="Vegfa"')

,gene_id,value_1,value_2,log2(fold_change),p_value,q_value
29378,Vegfa,7.99474,167.936,4.39272,0.00005,0.00031


In [11]:
# ポジコンである Mmp9 がとれるかの確認 (生物学的なQC)
deg_macrophage.query('`log2(fold_change)` > @threshold_fc & q_value < @threshold_qv').query('gene_id=="Mmp9"')

,gene_id,value_1,value_2,log2(fold_change),p_value,q_value
20560,Mmp9,0.088896,6.75021,6.24667,0.00005,0.00031


In [12]:
# 発現量（Expression, ex） の閾値
threshold_ex = 2

In [13]:
# とある条件で腫瘍組織内間質細胞で発現上昇する遺伝子をフィルタリング
deg_macrophage_tumor = deg_macrophage.query('value_2 > @threshold_ex & `log2(fold_change)` > @threshold_fc & q_value < @threshold_qv')

# 何行 (何遺伝子)
deg_macrophage_tumor.shape

(1921, 6)

------

------





## **テーブルデータ2：腫瘍組織で発現している遺伝子のリストを得る**


腫瘍細胞で発現する遺伝子のリストを読み込みます。
２つの列を使います。

- `trackig_id`: 遺伝子名
- `FPKM`: 遺伝子発現量（値が高いほどたくさんmRNAがある)

In [14]:
# 遺伝子発現量 (FPKM) をまとめた表を読み込む (Cufflinksの出力だとおもう)
url_fpkm_epi = "https://raw.githubusercontent.com/bioinfo-tsukuba/AdvancedCourse2021/main/3/data/CCCExplorer/EP_output/genes.read_group_tracking"

fpkm_epi = pd.read_csv(url_fpkm_epi, sep="\t")

In [15]:
# 行数、何数を調べる
fpkm_epi.shape

(184458, 9)

In [16]:
# データの頭の部分だけ確認
fpkm_epi.head()

,tracking_id,condition,replicate,raw_frags,internal_scaled_frags,external_scaled_frags,FPKM,effective_length,status
0,0610005C13Rik,q1,1,11.0,12.99970,12.99970,0.499583,-,OK
1,0610005C13Rik,q1,0,3.0,4.10373,4.10373,0.122142,-,OK
2,0610005C13Rik,q1,2,0.0,0.00000,0.00000,0.000000,-,OK
3,0610005C13Rik,q2,1,14.0,12.69620,12.69620,0.377883,-,OK
4,0610005C13Rik,q2,0,12.0,8.66261,8.66261,0.257830,-,OK


In [17]:
# 使いやすいように、使いたい列だけにする
fpkm_epi = fpkm_epi[['tracking_id', 'FPKM']]

fpkm_epi.head()

,tracking_id,FPKM
0,0610005C13Rik,0.499583
1,0610005C13Rik,0.122142
2,0610005C13Rik,0.000000
3,0610005C13Rik,0.377883
4,0610005C13Rik,0.257830


In [18]:
# とある条件で腫瘍細胞で発現する遺伝子をフィルタリング
fpkm_epi_tumor = fpkm_epi.query('FPKM > @threshold_ex')

fpkm_epi_tumor.shape

(72437, 2)

## **テーブルデータ３：ヒト-マウスのオーソログ関係の取得**

`data/CCCExplorer/HOM_MouseHumanSequence.rpt` は Mouse Genome Informatics database から取得されたヒト-マウスオーソログリスト

In [19]:
# データの読み込み
url_hom = 'https://raw.githubusercontent.com/bioinfo-tsukuba/AdvancedCourse2021/main/3/data/CCCExplorer/HOM_MouseHumanSequence.rpt'

dfhom = pd.read_csv(url_hom, sep="\t")

# 何行何列
dfhom.shape

(39522, 13)

In [20]:
# データの頭をみる
dfhom.head()

,HomoloGene ID,Common Organism Name,NCBI Taxon ID,Symbol,EntrezGene ID,Mouse MGI ID,HGNC ID,OMIM Gene ID,Genetic Location,"Genomic Coordinates (mouse: GRCm38, human: GRCh37.p10)",Nucleotide RefSeq IDs,Protein RefSeq IDs,SWISS_PROT IDs
0,3,"mouse, laboratory",10090,Acadm,11364,MGI:87867,NaN,NaN,Chr3 78.77 cM,Chr3:153922357-153944632(-),NM_007382,NP_031408,P45952
1,3,human,9606,ACADM,34,NaN,HGNC:89,607008.0,Chr1 p31,Chr1:76190043-76229355(+),"NM_000016,NM_001127328","NP_000007,NP_001120800,XP_005270868,XP_0052708...",P11310
2,5,"mouse, laboratory",10090,Acadvl,11370,MGI:895149,NaN,NaN,Chr11 42.96 cM,Chr11:70010183-70015411(-),NM_017366,NP_059062,P50544
3,5,human,9606,ACADVL,37,NaN,HGNC:92,609575.0,Chr17 p13.1,Chr17:7120444-7128586(+),"NM_000018,NM_001033859,NM_001270447,NM_001270448","NP_000009,NP_001029031,NP_001257376,NP_001257377",P49748
4,6,"mouse, laboratory",10090,Acat1,110446,MGI:87870,NaN,NaN,Chr9 29.12 cM,Chr9:53580522-53610382(-),NM_144784,NP_659033,Q8QZT1


In [21]:
# 使いやすいように、使いたい列だけにする
dfhom = dfhom[['HomoloGene ID', 'Symbol', 'Common Organism Name']]

# 確認
dfhom.head()

,HomoloGene ID,Symbol,Common Organism Name
0,3,Acadm,"mouse, laboratory"
1,3,ACADM,human
2,5,Acadvl,"mouse, laboratory"
3,5,ACADVL,human
4,6,Acat1,"mouse, laboratory"


## **テーブルデータ4：リガンド-受容体関係のデータ**

（`From` がリガンドで、 `To` が受容体）


In [22]:
# 読み込み
url_lr = 'https://raw.githubusercontent.com/bioinfo-tsukuba/AdvancedCourse2021/main/3/data/CCCExplorer/LR_manual_revised.txt'
dflr =  pd.read_csv(url_lr, sep='\t')

# 何行何列
dflr.shape

(1427, 2)

In [23]:
# 最初の５行
dflr.head()

,From,To
0,CCK,CCKAR
1,GAST,CCKBR
2,GRP,GRPR
3,IL17F,IL17RA
4,NTN1,DSCAM


## **データの読み込みのまとめ**

### テーブルデータ１：腫瘍組織内間質細胞で発現上昇する遺伝子のリスト `deg_macrophage_tumor`




In [24]:
# 何行 (何遺伝子)
print(deg_macrophage_tumor.shape)

# 最初の５行を表示
deg_macrophage_tumor.head()


(1921, 6)


,gene_id,value_1,value_2,log2(fold_change),p_value,q_value
15,0610010O12Rik,4.19646,8.98632,1.098560,0.00095,0.004437
21,0610031J06Rik,95.85410,144.76100,0.594760,0.00020,0.001113
30,0610040J01Rik,1.49871,3.08289,1.040560,0.00420,0.016218
36,1110002B05Rik,39.23150,69.53890,0.825809,0.00005,0.000310
39,1110003E01Rik,20.49960,34.26780,0.741253,0.00005,0.000310


### テーブルデータ２：腫瘍細胞において発現している遺伝子のリスト `fpkm_epi_tumor` 

In [25]:
# 何行何列
print(fpkm_epi_tumor.shape)

# 最初の５行を表示
fpkm_epi_tumor.head()

(72437, 2)


,tracking_id,FPKM
6,0610007C21Rik,46.5889
7,0610007C21Rik,63.4719
8,0610007C21Rik,80.0110
9,0610007C21Rik,59.6297
10,0610007C21Rik,49.4234


### テーブルデータ３：ヒト-マウスのオーソログ関係 `dfhom`

In [26]:
# 何行何列
print(dfhom.shape)

# 最初の５行を表示
dfhom.head()

(39522, 3)


,HomoloGene ID,Symbol,Common Organism Name
0,3,Acadm,"mouse, laboratory"
1,3,ACADM,human
2,5,Acadvl,"mouse, laboratory"
3,5,ACADVL,human
4,6,Acat1,"mouse, laboratory"


### テーブルデータ４：リガンド-受容体関係のデータ `dflr`

- `From` がリガンド遺伝子
- `To` が受容体遺伝子

In [27]:
# 何行何列
print(dflr.shape)

# 最初の５行を表示
dflr.head()

(1427, 2)


,From,To
0,CCK,CCKAR
1,GAST,CCKBR
2,GRP,GRPR
3,IL17F,IL17RA
4,NTN1,DSCAM


# **データの結合**

## **目標の確認**

- ほしいものは、「腫瘍組織内間質細胞で発現上昇しているリガンド遺伝子」と「腫瘍細胞において発現している受容体遺伝子」のペアのリストです。
- しかし、
  -  「腫瘍組織内間質細胞で発現上昇している遺伝子のリスト」と「腫瘍細胞において発現している受容体遺伝子のリスト」は、あくまでマウスの遺伝子です。
  - 「リガンド遺伝子と受容体遺伝子のペアのリスト」はヒトの遺伝子です。
- そこで、
  - 「腫瘍組織内間質細胞で発現上昇している遺伝子のリスト」と「腫瘍細胞において発現している受容体遺伝子のリスト」のそれぞれについて、対応するヒトのオーソログ遺伝子の情報をテーブルに追加できるとよさそうです。
  - その上で、「リガンド遺伝子と受容体遺伝子のペアのリスト」と結合すればよさそうです。
  - 「腫瘍組織内間質細胞で発現上昇している遺伝子のリスト」と「リガンド遺伝子と受容体遺伝子のペアのリスト」をリガンド遺伝子をキーに結合すると、「腫瘍組織内間質細胞で発現上昇しているリガンド遺伝子」が得られます。
  - 「腫瘍細胞において発現している遺伝子のリスト」と「リガンド遺伝子と受容体遺伝子のペアのリスト」を受容体遺伝子をキーに結合すると、「腫瘍細胞において発現している受容体遺伝子」が得られます。


### ヒント

以下の関数が便利です：

- `pd.merge()`
- `.query()`: 特定の条件の行を抽出する
- `.drop()` 特定の列を取り除く
- `.dropna()`: NA （欠損値）のある行を取り除く
- `.isin()` : Aの各要素がBに含まれるかをTRUE/FALSEのベクトルで返す 

## **目標１： ヒトとマウスのオーソログの対応が１対１になっている表がほしい**

`dfhom` 1対1オーソログだけでなく、1対多オーソログも含まれていたので、ちょっと面倒です。
このような場合、1対1オーソログのみに絞ることもありますし、全部残す場合もあります。ケースバイケース。

いずれにせよ、上の形では扱いづらいので、ヒトとマウスのオーソログの対応が1行ごとに並んだ形にしましょう。



- `dfhom` においてオーソログのID（ `HomoloGeneID` ） が同じ遺伝子名（ `Symbol` ）同士を同じ行にし、新たなデータフレーム `dfhom2` を作成します
  - ヒトは `Common Organism Name` が `human`、マウスは `mouse, laboratory` となっている
- `dfhom2` は３つの列を持ちます
  - `HomoloGeneID`: ある「オーソログ関係」に対するID
  - `mouse`:  `HomoloGeneID`列のIDに対応したマウスにおけるオーソログ遺伝子の名前
  - `human`:  `HomoloGeneID`列のIDに対応したヒトにおけるオーソログ遺伝子の名前



In [28]:
dfhom.head()

,HomoloGene ID,Symbol,Common Organism Name
0,3,Acadm,"mouse, laboratory"
1,3,ACADM,human
2,5,Acadvl,"mouse, laboratory"
3,5,ACADVL,human
4,6,Acat1,"mouse, laboratory"


In [29]:
# マウスの情報が入った行を抽出する例
dfhom.query(' `Common Organism Name` == "mouse, laboratory"')

,HomoloGene ID,Symbol,Common Organism Name
0,3,Acadm,"mouse, laboratory"
2,5,Acadvl,"mouse, laboratory"
4,6,Acat1,"mouse, laboratory"
6,7,Acvr1,"mouse, laboratory"
8,9,Sgca,"mouse, laboratory"
...,...,...,...
39517,133545,Gm5174,"mouse, laboratory"
39518,133545,Gm6713,"mouse, laboratory"
39519,133545,Gm6729,"mouse, laboratory"
39520,133547,Gm5792,"mouse, laboratory"


In [30]:
# マウスの行とヒトの表をそれぞれ抜き出し、 `outer_join` で joinしている
dfhom2 = pd.merge(
    dfhom.query('`Common Organism Name` == "mouse, laboratory"').drop(columns='Common Organism Name'),
    dfhom.query(' `Common Organism Name` == "human"').drop(columns='Common Organism Name'),
    how = "outer",
    on = "HomoloGene ID"
)

dfhom2.head()

,HomoloGene ID,Symbol_x,Symbol_y
0,3,Acadm,ACADM
1,5,Acadvl,ACADVL
2,6,Acat1,ACAT1
3,7,Acvr1,ACVR1
4,9,Sgca,SGCA


In [31]:
# 列名を変える
dfhom2 = dfhom2.rename(columns={"Symbol_x": "mouse", "Symbol_y": "human"})

dfhom2.head()

,HomoloGene ID,mouse,human
0,3,Acadm,ACADM
1,5,Acadvl,ACADVL
2,6,Acat1,ACAT1
3,7,Acvr1,ACVR1
4,9,Sgca,SGCA


## **目標２： 腫瘍組織内間質細胞において発現上昇している遺伝子の、ヒトにおけるオーソログ遺伝子のリストを得る**

遺伝子のリストとヒト-マウスオーソログの表を結合することで、マウスの遺伝子リストであってもヒトの遺伝子に関するデータベースの情報と照合することができるようになります。

そのために、JOIN をします。JOINは２つの表x, yそれぞれの列を比較し、x, yを結合した新しい表を作る操作です。

比較する列をキーと呼びます。表 x, y のキーがユニークで同一であれば単に横に連結すればいいですが、実際には、xもしくはyまたは両方で足りない/余分なキーがある場合があり、そのような場合にどう処理するかによって、JOINに様々な種類があります。

> join について https://qiita.com/matsuou1/items/b1bd9778610e3a586e71

- まず `deg_macrophage_tumor` と `dfhom2` を結合し、新たなデータフレーム `deg_macrophage_tumor_human` を作る
- 次に、`deg_macrophage_tumor_human` から NA (欠損値)のある行を除く

In [32]:
# ヒト-マウスオーソログ関係の表と遺伝子リスト表を結合する
## `left_join` は一番目の表（データフレーム) の行は全部残して、JOINを行う
deg_macrophage_tumor_human = pd.merge(
    deg_macrophage_tumor, 
    dfhom2, 
    how='left',
    left_on='gene_id',
    right_on='mouse'
)

# 行数と列数を確認する
print(deg_macrophage_tumor_human.shape)

# head
## deg_macrophage_tumor にあっても dfhom2 になかった列では、 `HomoloGene ID`やhumanが欠損値 (NA) になっている
deg_macrophage_tumor_human.head()

(1932, 9)


,gene_id,value_1,value_2,log2(fold_change),p_value,q_value,HomoloGene ID,mouse,human
0,0610010O12Rik,4.19646,8.98632,1.098560,0.00095,0.004437,NaN,NaN,NaN
1,0610031J06Rik,95.85410,144.76100,0.594760,0.00020,0.001113,10562.0,0610031J06Rik,C1orf85
2,0610040J01Rik,1.49871,3.08289,1.040560,0.00420,0.016218,49537.0,0610040J01Rik,C4orf19
3,1110002B05Rik,39.23150,69.53890,0.825809,0.00005,0.000310,NaN,NaN,NaN
4,1110003E01Rik,20.49960,34.26780,0.741253,0.00005,0.000310,NaN,NaN,NaN


In [33]:
# NA の無いの行だけにする
deg_macrophage_tumor_human = deg_macrophage_tumor_human.dropna()

# 行数と列数を確認する
print(deg_macrophage_tumor_human.shape)

# head
deg_macrophage_tumor_human.head()

(1674, 9)


,gene_id,value_1,value_2,log2(fold_change),p_value,q_value,HomoloGene ID,mouse,human
1,0610031J06Rik,95.854100,144.76100,0.594760,0.00020,0.001113,10562.0,0610031J06Rik,C1orf85
2,0610040J01Rik,1.498710,3.08289,1.040560,0.00420,0.016218,49537.0,0610040J01Rik,C4orf19
5,1110007C09Rik,22.560800,77.52120,1.780770,0.00005,0.000310,12269.0,1110007C09Rik,C9orf89
12,1700025G04Rik,1.123720,2.19438,0.965534,0.00350,0.013890,12776.0,1700025G04Rik,C1orf21
13,1700026D08Rik,0.584625,4.63796,2.987910,0.00005,0.000310,12605.0,1700026D08Rik,C15orf26


## **目標3: 腫瘍細胞において発現している遺伝子の、ヒトにおけるオーソログ遺伝子のリストを得る**

- `fpkm_epi_tumor` (腫瘍細胞において発現している遺伝子のリスト) と `dfhom2` (ヒトとマウスのオーソログ関係のリスト) を結合し、新たなデータフレーム `fpkm_epi_tumor_human` を作成する
- `fpkm_epi_tumor_human` でNA（欠損値）のある行を除く

In [34]:
# ヒト-マウスオーソログ関係の表と遺伝子リスト表を結合する
fpkm_epi_tumor_human = pd.merge(
    fpkm_epi_tumor, 
    dfhom2, 
    how='left',
    left_on='tracking_id',
    right_on='mouse'
)

# 行数と列数を確認する
print(fpkm_epi_tumor_human.shape)

# head
fpkm_epi_tumor_human.head()

(72726, 5)


,tracking_id,FPKM,HomoloGene ID,mouse,human
0,0610007C21Rik,46.5889,NaN,NaN,NaN
1,0610007C21Rik,63.4719,NaN,NaN,NaN
2,0610007C21Rik,80.0110,NaN,NaN,NaN
3,0610007C21Rik,59.6297,NaN,NaN,NaN
4,0610007C21Rik,49.4234,NaN,NaN,NaN


In [35]:
# NA が無いの行だけにする
fpkm_epi_tumor_human = fpkm_epi_tumor_human.dropna()

# 何行何列
print(fpkm_epi_tumor_human.shape)

# head
fpkm_epi_tumor_human.head()

(62264, 5)


,tracking_id,FPKM,HomoloGene ID,mouse,human
18,0610007P14Rik,23.5014,38284.0,0610007P14Rik,C14orf1
19,0610007P14Rik,33.9843,38284.0,0610007P14Rik,C14orf1
20,0610007P14Rik,24.8046,38284.0,0610007P14Rik,C14orf1
21,0610007P14Rik,25.0812,38284.0,0610007P14Rik,C14orf1
22,0610007P14Rik,21.5133,38284.0,0610007P14Rik,C14orf1


## **目標４:リガンド-受容体関係の情報に発現/発現変動遺伝子の情報を加える**

- `dflr` に、`ligand_up_in_tumor_macrophage` という列を追加する
  - `ligand_up_in_tumor_macrophage`列は、 `From`列のリガンドが `deg_macrophage_tumor_human` に含まれるなら `True`, 含まれないなら `False` が入る

- `dflr` に、`receptor_expressed_in_tumor_macrophage` という列を追加する
  - `receptor_expressed_in_tumor_macrophage`列は、 `To`列の受容体が `fpkm_epi_tumor_human` に含まれるなら `True`, 含まれないなら `False` が入る
- `dflr` から以下の条件に当てはまる行を抽出し、新たなデータフレーム `dflr_pair` を作成する
  - 条件: `ligand_up_in_tumor_macrophage`列 と  `receptor_expressed_in_tumor_macrophage`列 が両方 `True` である

In [36]:
# 何行何列
print(dflr.shape)

# ligand_up_in_tumor_macrophage という列を追加する。
## deg_macrophage_tumor_human に リガンドが含まれるならば TRUE 、そうでないならば FALSE となる
dflr['ligand_up_in_tumor_macrophage'] = dflr['From'].isin(deg_macrophage_tumor_human['human'])

# 何行何列
print(dflr.shape)

# head
dflr.head()

(1427, 2)
(1427, 3)


,From,To,ligand_up_in_tumor_macrophage
0,CCK,CCKAR,False
1,GAST,CCKBR,False
2,GRP,GRPR,False
3,IL17F,IL17RA,False
4,NTN1,DSCAM,False


In [37]:
# ligand_up_in_tumor_macrophage の数を集計
dflr.groupby('ligand_up_in_tumor_macrophage').size()

ligand_up_in_tumor_macrophage
False    1255
True      172
dtype: int64

In [38]:
# receptor_expressed_in_tumor_macrophage という列を追加する。
## fpkm_epi_tumor_human に 受容体が含まれるならばTRUE、そうでないならばFALSEとなる
dflr['receptor_expressed_in_tumor_macrophage'] = dflr['To'].isin(fpkm_epi_tumor_human['human'])

# 何行何列
print(dflr.shape)

# head
dflr.head()

(1427, 4)


,From,To,ligand_up_in_tumor_macrophage,receptor_expressed_in_tumor_macrophage
0,CCK,CCKAR,False,True
1,GAST,CCKBR,False,False
2,GRP,GRPR,False,False
3,IL17F,IL17RA,False,True
4,NTN1,DSCAM,False,False


In [39]:
# receptor_expressed_in_tumor_macrophage の数を集計
dflr.groupby('receptor_expressed_in_tumor_macrophage').size()

receptor_expressed_in_tumor_macrophage
False     426
True     1001
dtype: int64

In [40]:
# `ligand_up_in_tumor_macrophage` と  `receptor_expressed_in_tumor_macrophage` が両方TRUEである行を探す
dflr_pair = dflr.query('ligand_up_in_tumor_macrophage == True & receptor_expressed_in_tumor_macrophage == True')

# 何行何列
print(dflr_pair.shape)

# head
dflr_pair.head()

(129, 4)


,From,To,ligand_up_in_tumor_macrophage,receptor_expressed_in_tumor_macrophage
15,B2M,HFE,True,True
18,C3,ITGB2,True,True
20,C3,ITGAX,True,True
27,CREG1,IGF2R,True,True
55,GAS6,TYRO3,True,True


## **目標５：「先行研究で腫瘍組織内マクロファージで発現上昇が見られるリガンド」が見られるかを確認する** 

先行研究では、腫瘍組織内マクロファージで発現上昇が見られるリガンドが報告されている。

> "intratumoral macrophages express increased cathepsin K, COX-2, MMP-9, PDGF-B, uPA, VEGFA, and HGF (Wang et al., 2011)"
    - Wang et al., 2011 http://www.sciencedirect.com/science/article/pii/S0169500211002546

- 例えば、 `HGF` (Hepatocyte Growth Factor) が `dflr_pair` に含まれているかを確認する

In [41]:
dflr_pair.query('From == "HGF"')

,From,To,ligand_up_in_tumor_macrophage,receptor_expressed_in_tumor_macrophage
241,HGF,MET,True,True


----

----

----


# **本当の研究ではこの後どんなことをするか**

今のままでは、候補がかなり多いため、仮説を検証する実験をするのは難しそうです。また、この予測がどのくらい正しいのかも不明です。

元論文では、このリガンド-受容体ペアを足がかりに、以下のような解析をします。

- 受容体の下流のシグナル伝達経路から転写因子があるか探す（パスウェイデータベースを参照する）
- それらの下流の転写因子がターゲットとする遺伝子を探す（パスウェイデータベースや遺伝子制御関係のデータベースを参照する）
- それらのターゲット遺伝子群にランダムに比べて発現変動遺伝子が濃縮しているかを調べる（統計検定）

このような解析を加えることで、単にリガンドと受容体が共起だけでなく、さらに信頼性のあるペアを探すことができます。

ちなみに、著者らはこの方法をCCCExplorerと名付けています。詳しくは元の論文をご覧ください:

- Toi _et al_., Transcriptome Analysis of Individual Stromal Cell Populations Identifies Stroma-Tumor Crosstalk in Mouse Lung Cancer Model, Cell Reports (2015) http://dx.doi.org/10.1016/j.celrep.2015.01.040 


# **小まとめ**

このようにIDによる統合を繰り返すだけでもいろいろな仮説を立てられます。また、NGS解析とは言いつつ、NGSデータ以外の生命科学データを合わせることで、単なる発現変動遺伝子のリストよりも深い知見が得られるうることがみえてきたかと思います。
